<a href="https://colab.research.google.com/github/ChennyTech/TensorflowStudyExample/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        # nn.Sequential 컨테이너마냥 순서대로 담아주기만 하면 차례대로 실행됨
        self.layer1 = nn.Sequential(
            # input channel (rgb면 3), output channel(filter 수?), kernel = filter size (h,w) 
            # (int or tuple) int면 h = w
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            # (32-5)/1 +1 = 28 * 28 
            # output channel 수, channel 별로 normalization, spatial batch normalization
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            # 14 * 14
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            # 13+1 = 14
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            # 7 * 7
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 26250385.25it/s]                            


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


32768it [00:00, 460459.77it/s]
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


1654784it [00:00, 6457690.05it/s]                            
8192it [00:00, 170181.96it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw
Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!
Epoch [1/5], Step [100/600], Loss: 0.1343
Epoch [1/5], Step [200/600], Loss: 0.0799
Epoch [1/5], Step [300/600], Loss: 0.0383
Epoch [1/5], Step [400/600], Loss: 0.1113
Epoch [1/5], Step [500/600], Loss: 0.1027
Epoch [1/5], Step [600/600], Loss: 0.0136
Epoch [2/5], Step [100/600], Loss: 0.0490
Epoch [2/5], Step [200/600], Loss: 0.0449
Epoch [2/5], Step [300/600], Loss: 0.0615
Epoch [2/5], Step [400/600], Loss: 0.0389
Epoch [2/5], Step [500/600], Loss: 0.0200
Epoch [2/5], Step [600/600], Loss: 0.0102
Epoch [3/5], Step [100/600], Loss: 0.0261
Epoch [3/5], Step [200/600], Loss: 0.0146
Epoch [3/5], Step [300/600], Loss: 0.0233
Epoch [3/5], Step [400/600], Loss: 0.0403
Epoch [3/5], Step [500/600], Loss: 0.0163
Epoch [3/5], Step [600/600], Loss: 0.0671
Epoch [4/5], Step [100/600], Loss: 0.0024
Epoch [4/5], Step 

In [3]:
images.size()

torch.Size([100, 1, 28, 28])